# 08장 Dimensionality Reduction with Spark

- 차원축소는 unsupervised learning models 포함됨.
- regression, classification, clustering 등은 예측에 관심이 있지만, 차원축소는 특성치의 차원(변수개수)이 D개인 입력데이터셋에서 D개 보다 작은 k개의 차원(변수)으로 데이터들로 표현하는 것에 관심이 있음.
- 그러므로, <b><font color="red">데이터 전처리</font></b>와 <b><font color="red">특성치 변환</font></b>에 활용됨.

#### Dimensionality reduction 기술의 사용예
- 탐색적 데이터 분석
- 다른 머신러닝 모델 학습에 필요한 특성치를 추출
- 거대한 예측모델에서 저장공간이나 컴퓨팅 요구사항을 낮출때
- 아주 많은 수의 특성치를 가지고 있는 데이터에서 쉽게 학습모델을 만들기 위해서
- 예) 텍스트, 음성, 이미지, 비디오과 같이 고차원 데이터

#### 이장의 학습 목표
- MLlib에서 차원축소 models에 대한 소개
- 차원축소하기 적합하도록 얼굴이미지를 전처리 작업하기
- MLlib을 사용해서 차원축소 학습 시키기
- 결과를 시각화와 평가하기
- 차원축소 모델의 파라메터 선택하기

## 01절 Types of dimensionality reduction
- MLlib에서는 2가지 방식을 지원
- Principal Components Analysis (PCA, 주성분분석) 와 Singular Value Decomposition (SVD, 특이값 분해)


### Principal Components Analysis (주성분분석)
- PCA는 전체데이터에서 k개의 중요한 성분을 추출함.
- 이 주성분들은 서로 서로 관련성이 없음.
- 첫번째 주성분은 입력데이터에서 상당부분 분산을 설명할 수 있도록 계산되어짐.
- 각각의 하위 주성분은 순서적으로 많은 분산을 설명할 수 있도록 계산되어짐.
![PCA](pca_01.jpg "PCA")

### Singular Value Decomposition( 특이값 분해)
- SVD는 (m x n) 행렬 X 를 세개의 구성요소 행렬로 분해하는 것임.
- 첫번째 : (m x m) 행렬 U
- 두번째 : (m x n) 대각행렬 S,  S를 특이값라고 함.
- 세번째 : (n x n) 행렬 T
- X = U x S x V  로 표현함.
- 위의 공식으로는 차원축소가 실제적으로는 이루어지지 않아서, 특이값 S에서 상위 k개 만으로 표현함. => The truncated Singular Value Decomposition( 결단된 특이값 분해 )
![PCA](pca_02.jpg "PCA")
- top k 특이값과 PCA에서 top k 주성분은 매우 관련이 있음.

### Relationship with matrix factorization
- PCA와 SVD는 모두 행렬 인수분해(matrix factorization techniques)기법임.
- matrix factorization의 예는 4장 추천기에서 이미 나옴.

### Clustering as dimensionality reduction
- 이전장인 clustering model도  차원축소 방법으로 활용됨.

## 02절 Extracting the right features from your data

- 이번장에서는 Labeled Faces in the Wild (LFW) 데이터셋을 사용함.
- 이 데이터셋은 13,000개의 얼굴이미지를 포함하고, 이미지마다 사람의 이름이 적힘.


### Extracting features from the LFW dataset
- wget http://vis-www.cs.umass.edu/lfw/lfw-a.tgz  으로 데이터 다운로드
- tar xfvz lfw-a.tgz  압축을 품.
- lfw라는 디렉토리가 생기고, 그 하위디렉토리마다 한 사람씩의 얼굴이미지가 있음.

### Exploring the face data
- 차원축소 방법은 아주 계산적으로 비싼 작업이라서 메모리가 충분히 필요함.
- 실행방법 : bin/spark-shell --driver-memory 4g --master local[4]
- Spark는 텍스트파일과 hadoop input data를 읽는 방식을 지원하지만, 이미지와 같은 바이너리 데이터를 읽어오는 내장기능은 없음.
- Spark에서는 wholeTextFiles()함수가 있으며, 특정 디렉토리안에 있는 각각의 파일들의 위치를 접근할 수 있으며, 이를 이용하여 이미지들을 로딩하고 처리할 수 있는 코드를 작성할 수 있음.

In [ ]:
bin/spark-shell --driver-memory 4g --master local[4]

val path = "data/lfw/*"
val rdd = sc.wholeTextFiles(path)
val first = rdd.first
println(first)

- 위의 코드를 실행하면 볼 수 있는것 같이, wholeTextFiles()함수는 key-value 쌍을 포함한 RDD을 리턴함.
- key는 파일의 경로, value는 파일 전체 내용을 담고 있음.
- 경로명에 file:// 와 같이 필요없는 부분은 제거( file:// => 로컬파일시스템, hdfs:// =>하둡, s3n:// => 아마존S3 )

In [ ]:
val files = rdd.map { 
    case (fileName, content) => fileName.replace("file:", "") 
}
println(files.first)

println(files.count)

### Visualizing the face data

In [ ]:
path = "D:/Work_R/R-Project/99_ipython/01_ScalaML/lfw-a/lfw/Aaron_Eckhart/Aaron_Eckhart_0001.jpg"
ae = imread(path)
imshow(ae)

![PCA](lfw-a/lfw/Aaron_Eckhart/Aaron_Eckhart_0001.jpg "PCA")

### Extracting facial images as vectors

- 이미지는 각각의 픽셀의 위치를 x축, y축으로 표현되는 2차원 행렬로 표현
- 각각의 픽셀은 red, blue, green ( RGB )색상값을 갖음
- RGB 이미지를 grayscale(명암) images로 변환 필요.
- 2차원 행렬을 배열로 변환 필요
- 자바의 Abstract Window Toolkit (AWT)은 이미지 프로세싱 함수들을 지원
- java.awt 패키지안에 포함됨

#### Loading images

- 아래의 ImageIO.read()함수는 java.awt.image.BufferedImage 객체를 리턴
- 이 클래스는 이미지 데이터를 저장하고, 그외 유용한 함수들을 제공함

In [ ]:
import java.awt.image.BufferedImage
def loadImageFromFile(path: String): BufferedImage = {
    import javax.imageio.ImageIO
    import java.io.File
    ImageIO.read(new File(path))
}
    
val aePath = "data/lfw/Aaron_Eckhart/Aaron_Eckhart_0001.jpg"
val aeImage = loadImageFromFile(aePath)

- 이미지 정보가 표시되고, 폭과 높이가 250 pixel이고, RGB값으로 3개 색상을 갖음.

#### Converting to grayscale and resizing the images
- 아래 함수를 이미지들의 색상을 grayscale로 변환과 폭과 높이의 사이즈 조절함.
- grayscale로 변환과 resize가 필요한 이유는 효율성을 위함.
- RGB이고 250 X 250픽셀 1055개 이미지를 메모리에 로딩할려면, 3 x 250 x 250 x 1055 x 4 byte = 800 MB 가 필요
- grayscale이고 50 x 50 픽셀 1055개 이미지를 메모리에 로딩할려면, 1 x 50 x 50 x 1055 x 4 = 10MB 가 필요
- MLlib's PCA는 10,000개의 컬럼(특성치, 변수)이하가 적당하기 때문

In [ ]:
def processImage(image: BufferedImage, width: Int, height: Int): BufferedImage = {
    val bwImage = new BufferedImage(width, height, BufferedImage.TYPE_BYTE_GRAY)
    val g = bwImage.getGraphics()
    g.drawImage(image, 0, 0, width, height, null)
    g.dispose()
    bwImage
}
    
val grayImage = processImage(aeImage, 100, 100)
import javax.imageio.ImageIO
import java.io.File
ImageIO.write(grayImage, "jpg", new File("/tmp/aeGray.jpg"))

![PCA](aeGray.jpg "PCA")

#### Extracting feature vectors
- dimensionality reduction model 에서는 vector형식의 데이터를 input으로 사용함.
- 2차원 행렬을 배열로 변환이 필요하고 BufferedImage 클래스에서는 이 기능을 제공함.

In [ ]:
def getPixelsFromImage(image: BufferedImage): Array[Double] = {
   val width = image.getWidth
   val height = image.getHeight
   val pixels = Array.ofDim[Double](width * height)
   image.getData.getPixels(0, 0, width, height, pixels)
}

- 이미지를 읽어오고, 이미지를 변환하고 변환된 이미지에서 배열을 추출하는 3개의 함수를 이용해서  이미지 경로와 원하는 폭과 높이를 지정하면 배열형식( Array[Double] )의 데이터를 반환하는 함수를 작성
- Array[Double] 형식을 MLlib vector로 변환하고 성능을 높이기 위해서 변환된 RDD을 cache 처리함.

In [ ]:
def extractPixels(path: String, width: Int, height: Int): Array[Double] = {
    val raw = loadImageFromFile(path)
    val processed = processImage(raw, width, height)
    getPixelsFromImage(processed)
}
    
val pixels = files.map( f => extractPixels(f, 50, 50) )
println( 
    pixels.take(10).map(
        _.take(10).mkString("", ",", ", ...")
    ).mkString("\n")
)

import org.apache.spark.mllib.linalg.Vectors
val vectors = pixels.map(p => Vectors.dense(p))
vectors.setName("image-vectors")
vectors.cache


### Normalization
- dimensionality reduction models을 학습시키기 전에 입력데이터를 보통 정규화를 시킴.
- 이유는 Classification Model에 설명함.
- MLlib's feature 패키지안에 StandardScaler 클래스를 사용함.

In [ ]:
import org.apache.spark.mllib.linalg.Matrix
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import org.apache.spark.mllib.feature.StandardScaler

val scaler = new StandardScaler(withMean = true, withStd = false).fit(vectors)

val scaledVectors = vectors.map(v => scaler.transform(v))

- 조밀한(dense vector) 입력데이터(0 값이 거의 없음)에서 평균을 추출하는 작업하는 것을 주목이 필요.
- 그러나, 희소백터(0 값이 많음) 경우에는 평균을 추출하는 작업은 희소데이터를 조말한 데이터로 변환되기 때문에  차원이 많은 희소 입력데이터에 대해서 메모리를 과도하게 사용하므로 바람직하지 않음.

## 2절 Training a dimensionality reduction model

### Running PCA on the LFW dataset
- clustering에서는 RDD[Vector]을 입력데이터로 바로 사용하지만, PCA와 SVD에서는 RowMatrix 클래스의 멤버함수를 이용함.
- RowMatrix 객체를 생성하고 computePrincipalComponents()함수를 호출함.
- k는 주성분의 개수
- 아래와 같은 에러는 무시해도 됨.
- WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.
NativeSystemLAPACK
- WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK

In [ ]:
import org.apache.spark.mllib.linalg.Matrix
import org.apache.spark.mllib.linalg.distributed.RowMatrix
val matrix = new RowMatrix(scaledVectors)
val K = 10
val pc = matrix.computePrincipalComponents(K)

### Visualizing the Eigenfaces (고유 얼굴)
- 10개의 주성분은 원본 이미지의 대부분의 분산을 설명할 수 있는 latent (or hidden) features라고 생각할 수 있음.
- 얼굴인식과 이미지 프로세싱에서는 중요한 요소로 Eigenfaces을 자주 언급하고 PCA와 매우 관련이 있음.
- https://en.wikipedia.org/wiki/Eigenface 

In [ ]:
val rows = pc.numRows
val cols = pc.numCols
println(rows, cols)

- Breeze linear algebra library와 Python's numpy and matplotlib 을 사용해서 Eigenfaces을 시각화

In [ ]:
import breeze.linalg.DenseMatrix
val pcBreeze = new DenseMatrix(rows, cols, pc.toArray)

import breeze.linalg.csvwrite
csvwrite(new File("/tmp/pc.csv"), pcBreeze)

In [4]:
import numpy as np

pcs = np.loadtxt("D:/Work_R/R-Project/99_ipython/01_ScalaML/pc.csv", delimiter=",")
print(pcs.shape)

(2500, 10)


In [6]:
import matplotlib.pyplot as plt

def plot_gallery(images, h, w, n_row=2, n_col=5):
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[:, i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title("Eigenface %d" % (i + 1), size=12)
        plt.xticks(())
        plt.yticks(())
        
plot_gallery(pcs, 50, 50)

![PCA](pca_03.jpg "PCA")

### Interpreting the Eigenfaces
- PCA는 얼굴이미지의 다양한 특성치들을 표현하는 반복되는 패턴을 추출하는데 효과적임.
- 클러스터링모델과 비슷하게 각각의 주성분은 공통요소라고 해석할 수 있음.
- 각각의 주성분들이 어떤것을 나타내는지를 명확히 해석이 안 될때도 있음.
- 6, 9 번 이미지에서는 방향 요소를 추출
- 4, 5, 7, 10번 이미지에서는 머리패턴을 추출
- 1, 7, 9번 이미지에서는 눈, 코, 입의 얼굴요소를 추출

## 03절 Using a dimensionality reduction model
- dimensionality reduction의 궁극적인 목적은 raw 데이터셋에서 중요한 특성치나 변화를 찾아나는 좀더 단순한 표현임.
- 이를 위해서 학습된 모델을 이용해서 새롭고, 낮은 차원의 공간(더 적은 특성치 or 변수 or 컬럼)에서 표현할수 있도록 raw 데이터셋을 변환과정이 필요.
- 학습된 모델을 적용해보자.

### Projecting data using PCA on the LFW dataset
- 각각의 LFW 이미지를 10-차원 vector로 투영해보자.
- 이 과정은 주성분 행렬과 이미지 데이터 행렬을 곱셈 과정임.
- MLlib의 RowMatrix클래스의 multiply()함수를 이용함.

In [ ]:
val projected = matrix.multiply(pc)
println(projected.numRows, projected.numCols)

println(projected.rows.take(5).mkString("\n"))

### The relationship between PCA and SVD
- PCA와 SVD는 밀접한 관계를 갖음.
- PCA의 주성분과 SVD의 주성분(V 행렬)은 동일함.

In [ ]:
val svd = matrix.computeSVD(10, computeU = true)
println(s"U dimension: (${svd.U.numRows}, ${svd.U.numCols})")
println(s"S dimension: (${svd.s.size}, )")
println(s"V dimension: (${svd.V.numRows}, ${svd.V.numCols})")

def approxEqual(array1: Array[Double], array2: Array[Double], tolerance: Double = 1e-6): Boolean = {   
    val bools = array1.zip(array2).map {
        case (v1, v2) => 
            if(math.abs(math.abs(v1) - math.abs(v2)) > 1e-6) false 
            else true 
    }
    bools.fold(true)(_ & _)
}

println(approxEqual(Array(1.0, 2.0, 3.0), Array(1.0, 2.0, 3.0)))

println(approxEqual(Array(1.0, 2.0, 3.0), Array(3.0, 2.0, 1.0)))

println(approxEqual(svd.V.toArray, pc.toArray))

## 04절 Evaluating dimensionality reduction models
- PCA와 SVD는 결정론적 모델이고, 같은 input 데이터셋을 주어지면, 같은 결과가 나옴.
- PCA와 SVD의 k값을 증가시키면 모델의 성능이 좋아짐.( PCA와 SVD에서는 k성분이 증가하면 전체의 분산을 설명력이 높아짐 )
- 그러므로, k값에 따라서 설명력과 모델의 복잡도가 trade-off 관계임.

### Evaluating k for SVD on the LFW dataset
- SVD의 특이값(s)을 계산해보면, 실행한 결과 같이 내림차순으로 값이 나옴.
- SVD의 특이값(s)는 추가적인 변동을 얼마큼 설명하는지 나타냄.
- top 300개의 특이값을 뽑아서 그래프로 그려보자.

In [ ]:
val sValues = (1 to 5).map { 
    i => matrix.computeSVD(i, computeU = false).s 
}
sValues.foreach(println)

val svd300 = matrix.computeSVD(300, computeU = false)
val sMatrix = new DenseMatrix(1, 300, svd300.s.toArray)
csvwrite(new File("/tmp/s.csv"), sMatrix)

In [9]:
import numpy as np
import matplotlib.pyplot as plt

s = np.loadtxt("D:/Work_R/R-Project/99_ipython/01_ScalaML/s.csv", delimiter=",")
print(s.shape)
plt.plot(s)

(300,)


![PCA](pca_04.jpg "PCA")